# Day14_0 별첨: 이상탐지 (Anomaly Detection) - 정답

---

In [ ]:
# 환경 설정
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.datasets import make_blobs

import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

# 데이터 준비
n_normal = 500
X_normal, _ = make_blobs(n_samples=n_normal, centers=1, cluster_std=1.0, random_state=42)
n_outliers = 25
X_outliers = np.random.uniform(low=-6, high=6, size=(n_outliers, 2))

X = np.vstack([X_normal, X_outliers])
y_true = np.array([1] * n_normal + [-1] * n_outliers)

df = pd.DataFrame(X, columns=['Feature_1', 'Feature_2'])
df['Label'] = ['정상' if y == 1 else '이상' for y in y_true]

# 스케일링된 데이터 (One-Class SVM용)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("데이터 준비 완료!")
print(f"정상: {n_normal}개, 이상: {n_outliers}개")

---

## Q1. 기본 Isolation Forest ⭐

**문제**: Isolation Forest를 학습하고 예측된 이상치 개수를 출력하세요.

In [ ]:
# 정답 코드
# 1. Isolation Forest 생성
iso_forest = IsolationForest(random_state=42)

# 2. 학습 및 예측
y_pred = iso_forest.fit_predict(X)

# 3. 이상치 개수 출력
n_anomalies = np.sum(y_pred == -1)
print(f"예측된 이상치 개수: {n_anomalies}개")

In [ ]:
# 검증
assert n_anomalies > 0, "이상치가 탐지되어야 합니다"
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. IsolationForest() 생성
2. fit_predict()로 학습과 예측 동시 수행
3. -1이 이상, 1이 정상

**핵심 개념**:
- Isolation Forest는 unsupervised 학습
- 기본 contamination='auto' (데이터에서 추정)

**실무 팁**:
- 기본 설정으로 빠르게 시작 가능

---

## Q2. contamination 설정 ⭐

**문제**: contamination=0.10인 Isolation Forest를 학습하고 이상치 비율을 확인하세요.

In [ ]:
# 정답 코드
# 1. contamination=0.10 설정
iso_cont = IsolationForest(contamination=0.10, random_state=42)

# 2. 학습 및 예측
y_pred_cont = iso_cont.fit_predict(X)

# 3. 이상치 비율 계산
n_anomalies = np.sum(y_pred_cont == -1)
ratio = n_anomalies / len(X) * 100

print(f"이상치 개수: {n_anomalies}개")
print(f"이상치 비율: {ratio:.1f}%")

In [ ]:
# 검증
assert abs(ratio - 10.0) < 1.0, "비율이 약 10%여야 합니다"
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. contamination 파라미터로 예상 이상치 비율 설정
2. 실제 예측 비율 확인

**핵심 개념**:
- contamination: 이상치로 판단할 비율
- 0.10 = 전체의 10%를 이상으로 분류

**실무 팁**:
- 도메인 지식으로 contamination 설정

---

## Q3. 이상치 점수 확인 ⭐⭐

**문제**: decision_function으로 이상치 점수를 계산하고, 점수가 가장 낮은 5개 데이터를 출력하세요.

In [ ]:
# 정답 코드
# 1. 모델 학습
iso = IsolationForest(random_state=42)
iso.fit(X)

# 2. 이상치 점수 계산
scores = iso.decision_function(X)

# 3. 점수가 가장 낮은 5개
lowest_idx = np.argsort(scores)[:5]

print("=== 가장 이상한 5개 데이터 ===")
for idx in lowest_idx:
    print(f"인덱스 {idx}: 점수 = {scores[idx]:.4f}, 실제 = {'이상' if y_true[idx] == -1 else '정상'}")

In [ ]:
# 검증
assert len(lowest_idx) == 5
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. decision_function()으로 이상치 점수 계산
2. np.argsort()로 정렬된 인덱스 획득
3. 상위 5개 출력

**핵심 개념**:
- decision_function: 점수가 낮을수록 이상
- 음수: 이상, 양수: 정상

**실무 팁**:
- 점수로 이상치 우선순위화 가능

---

## Q4. 이상치 시각화 ⭐⭐

**문제**: Isolation Forest 결과를 Plotly 산점도로 시각화하세요.

In [ ]:
# 정답 코드
# 1. 예측
iso = IsolationForest(contamination=0.05, random_state=42)
y_pred = iso.fit_predict(X)

# 2. DataFrame에 결과 추가
df_viz = pd.DataFrame(X, columns=['Feature_1', 'Feature_2'])
df_viz['Prediction'] = ['정상' if p == 1 else '이상' for p in y_pred]

# 3. Plotly 시각화
fig = px.scatter(df_viz, x='Feature_1', y='Feature_2', color='Prediction',
                 title='Isolation Forest 이상탐지 결과',
                 color_discrete_map={'정상': 'blue', '이상': 'red'})
fig.update_traces(marker=dict(size=8))
fig.show()

In [ ]:
# 검증
assert 'Prediction' in df_viz.columns
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 예측 결과를 DataFrame에 추가
2. px.scatter로 색상 구분 시각화

**핵심 개념**:
- color_discrete_map으로 색상 지정
- 이상치는 빨간색으로 강조

**실무 팁**:
- 인터랙티브 시각화로 이상치 탐색

---

## Q5. One-Class SVM 기본 ⭐⭐

**문제**: One-Class SVM을 학습하고 예측 결과를 출력하세요.

In [ ]:
# 정답 코드
# 1. One-Class SVM 생성
ocsvm = OneClassSVM(kernel='rbf', nu=0.05, gamma='scale')

# 2. 스케일링된 데이터로 학습 및 예측
y_pred_ocsvm = ocsvm.fit_predict(X_scaled)

# 3. 결과 출력
n_normal = np.sum(y_pred_ocsvm == 1)
n_anomaly = np.sum(y_pred_ocsvm == -1)

print("=== One-Class SVM 결과 ===")
print(f"예측된 정상: {n_normal}개")
print(f"예측된 이상: {n_anomaly}개")

In [ ]:
# 검증
assert n_anomaly > 0, "이상치가 탐지되어야 합니다"
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. OneClassSVM 생성 (nu로 이상 비율 제어)
2. 스케일링된 데이터 사용 (필수!)

**핵심 개념**:
- nu: 이상치 비율의 상한
- gamma='scale': 자동 감마 설정

**실무 팁**:
- One-Class SVM은 스케일링 필수

---

## Q6. nu 파라미터 실험 ⭐⭐⭐

**문제**: nu=0.01, 0.05, 0.10 세 가지 값으로 One-Class SVM을 학습하고 이상치 개수를 비교하세요.

In [ ]:
# 정답 코드
nu_values = [0.01, 0.05, 0.10]
nu_results = []

for nu in nu_values:
    ocsvm = OneClassSVM(kernel='rbf', nu=nu, gamma='scale')
    y_pred = ocsvm.fit_predict(X_scaled)
    
    n_anomaly = np.sum(y_pred == -1)
    ratio = n_anomaly / len(X) * 100
    
    nu_results.append({'nu': nu, 'Anomalies': n_anomaly, 'Ratio (%)': ratio})

nu_df = pd.DataFrame(nu_results)
print("=== nu에 따른 이상치 개수 ===")
print(nu_df.to_string(index=False))

In [ ]:
# 검증
assert len(nu_df) == 3
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 여러 nu 값으로 실험
2. 이상치 개수와 비율 계산

**핵심 개념**:
- nu는 이상치 비율의 상한 (상한이므로 실제는 더 적을 수 있음)
- nu가 클수록 더 많은 데이터를 이상으로 판단

**실무 팁**:
- nu 값 실험으로 적절한 임계값 탐색

---

## Q7. 성능 평가 ⭐⭐⭐

**문제**: y_true와 예측 결과를 비교하여 Accuracy, Precision, Recall을 계산하세요.

In [ ]:
# 정답 코드
# 1. Isolation Forest 예측
iso = IsolationForest(contamination=0.05, random_state=42)
y_pred_iso = iso.fit_predict(X)

# 2. 성능 평가 (이상을 positive로)
acc = accuracy_score(y_true, y_pred_iso)
prec = precision_score(y_true, y_pred_iso, pos_label=-1)
rec = recall_score(y_true, y_pred_iso, pos_label=-1)
f1 = f1_score(y_true, y_pred_iso, pos_label=-1)

print("=== 성능 평가 ===")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 Score:  {f1:.4f}")

In [ ]:
# 검증
assert acc > 0.8, "Accuracy가 0.8 이상이어야 합니다"
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. sklearn.metrics 함수 사용
2. pos_label=-1로 이상을 positive로 설정

**핵심 개념**:
- 이상탐지에서 이상(-1)을 positive로 설정
- Recall: 실제 이상 중 탐지된 비율
- Precision: 탐지된 것 중 실제 이상 비율

**실무 팁**:
- 불균형 데이터에서 F1이 중요 (Accuracy 신뢰 X)

---

## Q8. contamination 최적화 ⭐⭐⭐⭐

**문제**: contamination을 0.01~0.20까지 변화시키며 F1 스코어가 가장 높은 값을 찾으세요.

In [ ]:
# 정답 코드
contamination_range = np.arange(0.01, 0.21, 0.01)
best_f1 = 0
best_cont = 0
results = []

for cont in contamination_range:
    iso = IsolationForest(contamination=cont, random_state=42)
    y_pred = iso.fit_predict(X)
    
    f1 = f1_score(y_true, y_pred, pos_label=-1)
    results.append({'contamination': cont, 'F1': f1})
    
    if f1 > best_f1:
        best_f1 = f1
        best_cont = cont

print(f"최적 contamination: {best_cont:.2f}")
print(f"최고 F1 Score: {best_f1:.4f}")

# 시각화
results_df = pd.DataFrame(results)
fig = px.line(results_df, x='contamination', y='F1', markers=True,
              title='Contamination에 따른 F1 Score 변화')
fig.add_vline(x=best_cont, line_dash='dash', line_color='red')
fig.show()

In [ ]:
# 검증
assert best_f1 > 0.5, "F1이 0.5 이상이어야 합니다"
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 여러 contamination 값 탐색
2. F1 스코어 기준 최적값 선택
3. 시각화로 확인

**핵심 개념**:
- contamination이 너무 낮으면 이상 놓침 (Recall 감소)
- contamination이 너무 높으면 오탐 증가 (Precision 감소)

**실무 팁**:
- 레이블이 있으면 최적화 가능
- 없으면 도메인 지식으로 설정

---

## Q9. 모델 비교 ⭐⭐⭐⭐

**문제**: Isolation Forest와 One-Class SVM의 성능을 DataFrame으로 비교하세요.

In [ ]:
# 정답 코드
# 1. Isolation Forest
iso = IsolationForest(contamination=0.05, random_state=42)
y_pred_iso = iso.fit_predict(X)

# 2. One-Class SVM
ocsvm = OneClassSVM(kernel='rbf', nu=0.05, gamma='scale')
y_pred_ocsvm = ocsvm.fit_predict(X_scaled)

# 3. 성능 비교
models_comparison = []

for name, y_pred in [('Isolation Forest', y_pred_iso), ('One-Class SVM', y_pred_ocsvm)]:
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, pos_label=-1)
    rec = recall_score(y_true, y_pred, pos_label=-1)
    f1 = f1_score(y_true, y_pred, pos_label=-1)
    
    models_comparison.append({
        'Model': name,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1': f1
    })

comparison_df = pd.DataFrame(models_comparison)
print("=== 모델 비교 ===")
print(comparison_df.to_string(index=False))

In [ ]:
# 검증
assert len(comparison_df) == 2
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 동일 조건으로 두 모델 학습
2. 네 가지 지표 계산
3. DataFrame으로 정리

**핵심 개념**:
- 공정한 비교를 위해 동일 조건 설정
- F1 스코어로 최종 판단

**실무 팁**:
- 데이터 특성에 따라 적합한 모델 선택

---

## Q10. 종합 시각화 ⭐⭐⭐⭐⭐

**문제**: 두 모델의 결과를 make_subplots로 나란히 시각화하고, 실제 레이블과 비교하세요.

In [ ]:
# 정답 코드
# 1. 세 개의 서브플롯 생성
fig = make_subplots(rows=1, cols=3, subplot_titles=['실제 레이블', 'Isolation Forest', 'One-Class SVM'])

# 2. 실제 레이블
fig.add_trace(
    go.Scatter(x=X[y_true==1, 0], y=X[y_true==1, 1], mode='markers',
               marker=dict(color='blue', size=6), name='정상', showlegend=True),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=X[y_true==-1, 0], y=X[y_true==-1, 1], mode='markers',
               marker=dict(color='red', size=8, symbol='x'), name='이상', showlegend=True),
    row=1, col=1
)

# 3. Isolation Forest
fig.add_trace(
    go.Scatter(x=X[y_pred_iso==1, 0], y=X[y_pred_iso==1, 1], mode='markers',
               marker=dict(color='blue', size=6), showlegend=False),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=X[y_pred_iso==-1, 0], y=X[y_pred_iso==-1, 1], mode='markers',
               marker=dict(color='red', size=8, symbol='x'), showlegend=False),
    row=1, col=2
)

# 4. One-Class SVM
fig.add_trace(
    go.Scatter(x=X[y_pred_ocsvm==1, 0], y=X[y_pred_ocsvm==1, 1], mode='markers',
               marker=dict(color='blue', size=6), showlegend=False),
    row=1, col=3
)
fig.add_trace(
    go.Scatter(x=X[y_pred_ocsvm==-1, 0], y=X[y_pred_ocsvm==-1, 1], mode='markers',
               marker=dict(color='red', size=8, symbol='x'), showlegend=False),
    row=1, col=3
)

fig.update_layout(title='이상탐지 모델 비교', height=400)
fig.show()

In [ ]:
# 검증
print("시각화 완료!")
print("검증 통과!")

### 💡 풀이 설명

**접근 방법**:
1. make_subplots로 3개 플롯 생성
2. 실제 레이블, IF 예측, OCSVM 예측 비교
3. 동일 색상/기호로 일관성 유지

**핵심 개념**:
- 시각적 비교로 모델 특성 파악
- 실제 레이블과 비교하여 정확도 확인

**실무 팁**:
- 여러 모델 결과를 병렬 비교하여 최적 모델 선택
- 오탐(False Positive)과 누락(False Negative) 패턴 분석

---

## 📊 학습 정리

### Part 1: 이상탐지 기초 요약

| 개념 | 설명 | 핵심 포인트 |
|------|------|------------|
| 이상탐지 | 비정상 데이터 찾기 | 극심한 불균형 문제 해결 |
| Global Outlier | 전체 분포에서 이상 | 가장 일반적 |
| Isolation Forest | 고립 기반 탐지 | 경로가 짧으면 이상 |
| contamination | 예상 이상치 비율 | 도메인 지식 필요 |

### Part 2: 이상탐지 활용 요약

| 모델 | 특징 | 추천 상황 |
|------|------|----------|
| Isolation Forest | 빠름, 해석 가능 | 고차원, 대용량 |
| One-Class SVM | 정상 경계 학습 | 정상 데이터만 있을 때 |

### 💡 실무 팁

1. **contamination 신중히**: 도메인 지식 기반으로 설정
2. **스케일링 확인**: One-Class SVM은 스케일링 필수
3. **decision_function 활용**: 점수로 우선순위화 가능
4. **F1 스코어 중시**: 불균형 데이터에서 Accuracy는 무의미
5. **실시간 모니터링**: Isolation Forest가 빠르고 효율적